In [ ]:

from classes.event_graph import EventGraph
from classes.event_node import EventNode
from data.event_types import event_types

n0 = EventNode(event_types[0]) 
n1 = EventNode(event_types[1]) 
n2 = EventNode(event_types[2])

print("----------------------------")

eg = EventGraph()
eg.add_node(n0)
eg.add_node(n1)
eg.add_node(n2)
eg.add_edge(n0, n1)
eg.add_edge(n1, n2)
eg.visualize()

In [ ]:
import hashlib

def stable_hash(label: str):
    return hashlib.blake2b(label.encode("utf-8"), digest_size=16).hexdigest()

def WL_neighborhood_labels(graph: EventGraph, iterations: int, inspect: bool = False):
    labels = {node: graph.nodes[node].event_type for node in graph.nodes}
    
    for _ in range(iterations):
        new_labels = {}
        for node in graph.nodes:
            nbrs = graph.adj_list.get(node, [])  # safe default
            neighbor_labels = [labels[nbr] for nbr in nbrs if nbr in labels]
            
            # deterministic sort even if labels later become non-strings
            neighbor_labels.sort(key=repr)

            signature = (labels[node], tuple(neighbor_labels))

            if inspect:
                # keep inspection output readable and still a string so next iter is safe
                new_label = signature
            else:
                new_label = stable_hash(repr(signature))
            
            new_labels[node] = new_label
            
        labels = new_labels
        
    return labels

In [ ]:
labels = WL_neighborhood_labels(eg, 3)
print(labels)

{'6fd6806e29df41b381c4199080337494': ((('started_school', ('finished_school',)), (('finished_school', ('started_work',)),)), ((('finished_school', ('started_work',)), (('started_work', ()),)),)), '8c5ecb92f3ba4b7d9e5717157ae1d52b': ((('finished_school', ('started_work',)), (('started_work', ()),)), ((('started_work', ()), ()),)), '7a17f597ef1c4ff1a96dde2c8307b5a2': ((('started_work', ()), ()), ())}
